In [1]:
import os
import pandas as pd
from rich import print as rprint

# Import util functions
from helper import set_openai_key,\
                    test_openai_api,\
                    create_openai_client,\
                    print_pretty,\
                    generate_function_schema

# Import tools
from helper import get_directory_tree,\
                    get_lines_from_file,\
                    search_code, \
                    get_object_definition

print("Import successful!")

Import successful!


In [2]:
set_openai_key()
#test_openai_api()

API key set successfully.


In [3]:
# Define paths
comp_dir = "konwinski-prize"
comp_kaggle_evaluation_dir = os.path.join(comp_dir, "kaggle_evaluation")
comp_kprize_setup_dir = os.path.join(comp_dir, "kprize_setup")

comp_data_zip_path = os.path.join(comp_dir, "data.a_zip")
comp_data_dir = os.path.join(comp_dir, "data")
comp_data_parquet_path = os.path.join(comp_data_dir, "data.parquet")
comp_conda_packages_dir = os.path.join(comp_data_dir, "conda_packages")
comp_pip_packages_dir = os.path.join(comp_data_dir, "pip_packages")
comp_repo_configs_dir = os.path.join(comp_data_dir, "repo_configs")
comp_repos_dir = os.path.join(comp_data_dir, "repos")

In [4]:
# Load dataset
kprize_df = pd.read_parquet(comp_data_parquet_path)
kprize_df.head()

,instance_id,repo,problem_statement,patch,test_patch,pull_number,base_commit,PASS_TO_PASS,FAIL_TO_PASS,issue_numbers
0,pylint-dev__astroid-2496,pylint-dev/astroid,TypeError: unsupported format string passed to...,diff --git a/ChangeLog b/ChangeLog\nindex 4560...,diff --git a/tests/test_inference.py b/tests/t...,2496,8d3cdbbe6685fd8cf211816bec56c90f38f1859e,[tests/test_inference.py::InferenceUtilsTest::...,[tests/test_inference.py::test_formatted_fstri...,[2492]
1,pylint-dev__astroid-2468,pylint-dev/astroid,Pylint checks against incorrect type with prop...,diff --git a/ChangeLog b/ChangeLog\nindex fdbb...,diff --git a/tests/test_inference.py b/tests/t...,2468,6db3a60553ff538a936d5dda23d67a3924a57f45,[tests/test_inference.py::InferenceUtilsTest::...,[tests/test_inference.py::InferenceTest::test_...,[2467]
2,astropy__astropy-17048,astropy/astropy,QTable cannot take `dimensionless_unscaled` wh...,diff --git a/astropy/table/table.py b/astropy/...,diff --git a/astropy/table/tests/test_table.py...,17048,d60f6b72cd525262bfd179331d9fe4474177918f,[astropy/table/tests/test_table.py::TestSetTab...,[astropy/table/tests/test_table.py::test_qtabl...,[17047]
3,astropy__astropy-16898,astropy/astropy,BUG: tables do not deal well with zero-sized s...,diff --git a/astropy/io/registry/core.py b/ast...,diff --git a/astropy/io/fits/tests/test_connec...,16898,ee6d087baf301c1d08db92e6e5b6d909d57e6fac,[astropy/io/fits/tests/test_connect.py::TestSi...,[astropy/io/fits/tests/test_connect.py::test_z...,[16897]
4,astropy__astropy-16830,astropy/astropy,KeyError: 'version_1_3_or_later' when parsing ...,diff --git a/astropy/io/votable/tree.py b/astr...,diff --git a/astropy/io/votable/tests/test_tre...,16830,e39f486fec48d87aa3677326167954370d7a7bf9,[astropy/io/votable/tests/test_tree.py::test_c...,[astropy/io/votable/tests/test_tree.py::test_v...,"[16825, 16826]"


In [5]:
idx = 2
row = kprize_df.iloc[idx]
problem_statement = row["problem_statement"]
instance_id = row["instance_id"]
p2p_tests = row["PASS_TO_PASS"]
f2p_tests = row["FAIL_TO_PASS"]
repo_path = os.path.join(comp_repos_dir, f'repo__{instance_id}')

### Util Functions

In [6]:
# Generate function json schema
def example_function(a: int, b: str = "default", c: bool = True):
    """Example function with defaults."""
    pass
    
generate_function_schema(example_function)

{'name': 'example_function',
 'description': 'Example function with defaults.',
 'parameters': {'type': 'object',
  'properties': {'a': {'type': 'integer'},
   'b': {'type': 'string'},
   'c': {'type': 'boolean'}},
  'required': ['a']}}

### Tools

### Search & Discovery Tools

In [7]:
# Display tree
tree = get_directory_tree(repo_path)
#rprint(tree)

In [8]:
# Get code snippet from line range
code = get_lines_from_file(file_path = os.path.join(repo_path, "astropy/_dev/scm_version.py"),
                                           line_range="1-20")
print(code)

# Try to use setuptools_scm to get the current version; this is only used
# in development installations from the git repository.
from pathlib import Path

try:
    from setuptools_scm import get_version

    version = get_version(root=Path(__file__).parents[1], relative_to=__file__)
except Exception:
    raise ImportError("setuptools_scm broken or not installed")



In [9]:
# Search code by string
res = search_code(root_directory=repo_path,
    search_string="normalize_kernel",
    n_lines_before=2,
    n_lines_after=2,
    return_imports=True
)

print_pretty(res[:2])

[
    {
        "content": "                image, psf, boundary=\"extend\", normalize_kernel=True",
        "context_after": [
            "            )",
            "            randomImage = np.random.normal(0, 2, image.shape)"
        ],
        "context_before": [
            "        def convolve_with_noise(image, psf):",
            "            convolvedImage = convolve("
        ],
        "file": "konwinski-prize/data/repos/repo__astropy__astropy-17048/astropy/visualization/tests/test_lupton_rgb.py",
        "imports": [
            "from astropy.convolution import Gaussian2DKernel, convolve",
            "from astropy.utils.compat.optional_deps import HAS_MATPLOTLIB",
            "from astropy.visualization import lupton_rgb",
            "from astropy.visualization.interval import ManualInterval",
            "from astropy.visualization.stretch import LinearStretch",
            "from numpy.testing import assert_allclose, assert_equal",
            "import matplotlib.pypl

In [10]:
# Get Class definition
res = get_object_definition(
    root_directory=repo_path,
    object_name="Gaussian1DKernel",
    return_imports=True
)

print_pretty(res)

{
    "content": "class Gaussian1DKernel(Kernel1D):",
    "definition_block": [
        "class Gaussian1DKernel(Kernel1D):",
        "    \"\"\"",
        "    1D Gaussian filter kernel.",
        "",
        "    The Gaussian filter is a filter with great smoothing properties. It is",
        "    isotropic and does not produce artifacts.",
        "",
        "    The generated kernel is normalized so that it integrates to 1.",
        "",
        "    Parameters",
        "    ----------",
        "    stddev : number",
        "        Standard deviation of the Gaussian kernel.",
        "    x_size : int, optional",
        "        Size of the kernel array. Default = \u230a8*stddev+1\u230b.",
        "    mode : {'center', 'linear_interp', 'oversample', 'integrate'}, optional",
        "        One of the following discretization modes:",
        "            * 'center' (default)",
        "                Discretize model by taking the value",
        "                at the cen